<a href="https://colab.research.google.com/github/mohammad-hosein/Semi-Blind-MLP/blob/main/Copy_of_Semi_blind_MLP_Paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import math
import pandas as pd
import tensorflow as tf
import random
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier

# Generating datasets and preprocessing

In [2]:
'''Fading channel for specific number of antennas at transmitter and receiver side'''
j = complex(0, 1) #Imaginary part
def fading_channel (Nt, Nr):
  h_real = np.random.normal(0, 0.5, size = (Nr, Nt))
  h_img = np.random.normal(0, 0.5, size = (Nr, Nt))
  H_Rayleigh = np.array(h_real + h_img*j, dtype = complex)
  
  return H_Rayleigh

In [3]:
'''SSK Constellation set'''
Constellation_set= np.zeros(shape= (4, 4))
Constellation_set[0] = np.array([1, 0, 0, 0])
Constellation_set[1] = np.array([0, 1, 0, 0])
Constellation_set[2] = np.array([0, 0, 1, 0])
Constellation_set[3] = np.array([0, 0, 0, 1])

In [4]:
H_Rayleigh = fading_channel (4,2)

In [5]:
'''Complex noise'''
def noise_complex (N0, Nr):
  n_real = np.random.normal(0, N0, size = Nr)
  n_img=np.random.normal(0, N0, size = Nr)
  noise_complex=np.array(n_real + n_img*j, dtype = complex)
  
  return noise_complex

In [6]:
'''Creating pilot signals'''
def training_pilots_at_receiver (Label):
  i = 0
  
  y_receiver = np.zeros(shape = (4, 2), dtype = "complex_")
  for i in range(Label):
    y_receiver[i] = np.dot(H_Rayleigh, Constellation_set[i], out = None) + noise_complex (0.1, 2)
    i += 1

  y_receiver_real = y_receiver.real
  y_receiver_imag = y_receiver.imag
  y_receiver = np.concatenate((y_receiver_real, y_receiver_imag), axis = 1)
  y_receiver_complex = pd.DataFrame(y_receiver, columns=['Ant1_real', 'Ant2_real', 'Ant1_imag', 'Ant2_imag'])
  y_receiver_complex['Label'] = np.arange(0, 4) 

  return y_receiver_complex

In [7]:
training_pilots_at_receiver = training_pilots_at_receiver (4)

In [8]:
'''Creating test signals'''
def test_ (packet_length):
  #Define an empty dataframe
  y_receive_test = pd.DataFrame()

  for i in range(packet_length):
    #Generating received signals
    y_one_receive_signal = np.zeros(shape = (1, 2), dtype = "complex_")
    
    #Choosing a random constellation point
    rand = random.randrange(0, 4, 1)
    
    #Y = HX+N 
    y_one_receive_signal = np.dot(H_Rayleigh, Constellation_set[rand], out = None) + noise_complex (0.1, 2)
    
    #Transforming complex signal to a two dimension integer
    y_one_receive_signal_real = y_one_receive_signal.real
    y_one_receive_signal_imag = y_one_receive_signal.imag
    y_receive = np.concatenate((y_one_receive_signal_real,y_one_receive_signal_imag), axis = None)
    
    #Filling dataframe rows
    y_receive_without_label = pd.DataFrame(y_receive.reshape((1, 4)), columns = ['Ant1_real', 'Ant2_real', 'Ant1_imag', 'Ant2_imag'])
    
    #Assigning related label to received pilots
    if rand == 0 :
      y_receive_without_label['Label']= 0
    elif rand == 1 :
      y_receive_without_label['Label']= 1
    elif rand == 2 :
      y_receive_without_label['Label']= 2
    elif rand == 3 :
      y_receive_without_label['Label']= 3
    
    #Join all test signals to data set
    y_receive_test = pd.concat([y_receive_test, y_receive_without_label])

  return y_receive_test

In [9]:
def test_label (KNN_or_NN): #Depend on model
  test = test_ (2000)

  #Test data for KNN
  if KNN_or_NN == 1 :
    test_features = test[['Ant1_real', 'Ant2_real', 'Ant1_imag', 'Ant2_imag']]
    test_label = test['Label']
    return test_features, test_label

  #Test data for NN
  elif KNN_or_NN == 0:
    test_features = test[['Ant1_real', 'Ant2_real', 'Ant1_imag', 'Ant2_imag']]
    test_label = test['Label']
    test_label_categorical = to_categorical(test_label, num_classes = 4)
    return test_features, test_label_categorical

In [10]:
test_label (1)

(    Ant1_real  Ant2_real  Ant1_imag  Ant2_imag
 0   -0.099294   0.708766  -0.179159  -0.685387
 0    0.435817   0.383128   0.198074   0.051889
 0    0.308560  -0.235767   0.231946   0.471874
 0   -0.105692   0.508818  -0.158930  -0.679716
 0    0.215384   0.344180  -0.089760   0.075243
 ..        ...        ...        ...        ...
 0    0.089977   0.574765  -0.154180  -0.571171
 0    0.439689   0.418626  -0.064513  -0.045399
 0    0.602776   0.545105   0.114216   0.096548
 0    0.171038   0.324386  -0.008940   0.121382
 0   -0.203006   0.849035  -0.281237  -0.694930
 
 [2000 rows x 4 columns], 0    2
 0    3
 0    0
 0    2
 0    3
     ..
 0    2
 0    3
 0    3
 0    3
 0    2
 Name: Label, Length: 2000, dtype: int64)

In [11]:
test_label (0)

(    Ant1_real  Ant2_real  Ant1_imag  Ant2_imag
 0    0.300557   0.470942   0.106826   0.300141
 0    0.195874   0.191029   0.789898  -0.255384
 0   -0.212535   0.672780  -0.263801  -0.762225
 0   -0.056447   0.837857  -0.131285  -0.797899
 0    0.194243   0.261004   0.976005  -0.323597
 ..        ...        ...        ...        ...
 0   -0.009293   0.553809  -0.083532  -0.564354
 0    0.025323   0.831427  -0.239296  -0.655446
 0   -0.039351   0.654494  -0.367115  -0.631083
 0    0.440962   0.292328   0.920044  -0.373680
 0   -0.096135   0.708343  -0.240253  -0.756861
 
 [2000 rows x 4 columns], array([[0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        ...,
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]], dtype=float32))

In [12]:
x_test_svm, y_test_svm = test_label (1)
x_test_nn, y_test_nn = test_label (0)


# Generating virtual pilots (data augmentation)

In [13]:
'''Noise for data augmentation'''
def noise_real (N0,Nr):
  noise_real=np.random.normal(0, N0, size=Nr)
  
  return noise_real

In [14]:
def data_augmentation (sample_size, noise_variance):
  #Generating virtual pilots based on actual pilots
  augmented_data = pd.DataFrame()

  for i in range(sample_size):
    frame_augmented = training_pilots_at_receiver[['Ant1_real', 'Ant2_real', 'Ant1_imag', 'Ant2_imag']] + noise_real(noise_variance, 4)
    frame_augmented['Label'] = np.arange(0, 4)
    augmented_data = augmented_data.append(frame_augmented)
    i += 1

  return augmented_data

In [15]:
data_augmentation = data_augmentation(50, 0.03)

In [16]:
print(data_augmentation.describe())
print(training_pilots_at_receiver.describe())

        Ant1_real   Ant2_real   Ant1_imag   Ant2_imag      Label
count  200.000000  200.000000  200.000000  200.000000  200.00000
mean     0.132527    0.263313    0.281909   -0.152827    1.50000
std      0.228139    0.233171    0.446038    0.450185    1.12084
min     -0.292560   -0.105619   -0.295815   -0.885362    0.00000
25%     -0.049733    0.114037   -0.083915   -0.491381    0.75000
50%      0.140340    0.238990    0.177482   -0.124142    1.50000
75%      0.274269    0.386964    0.534047    0.200408    2.25000
max      0.508384    0.683499    1.031453    0.466322    3.00000
       Ant1_real  Ant2_real  Ant1_imag  Ant2_imag     Label
count   4.000000   4.000000   4.000000   4.000000  4.000000
mean    0.138103   0.265526   0.286552  -0.151789  1.500000
std     0.259640   0.266350   0.512470   0.517231  1.290994
min    -0.178235  -0.031945  -0.208354  -0.789444  0.000000
25%     0.021074   0.151980  -0.033233  -0.435901  0.750000
50%     0.141007   0.239899   0.191384  -0.109100  1.50

# Split dataset to train and test

In [17]:
'''Spliting features and labels'''
#xy=pd.concat([l,q1])       #Enable when you want vp for SVM & KNN
xy = training_pilots_at_receiver                      #Enable when you dont want vp for SVM & KNN
xy_NN = pd.concat([data_augmentation, training_pilots_at_receiver])     #Enable when you want vp for NN
#xy_NN=q1                   #Enable when you dont want vp for NN

In [18]:
xy

,Ant1_real,Ant2_real,Ant1_imag,Ant2_imag,Label
0,0.087510,-0.031945,0.357627,0.400489,0
1,0.194505,0.266510,0.971796,-0.318054,1
2,-0.178235,0.614249,-0.208354,-0.789444,2
3,0.448632,0.213288,0.025141,0.099854,3


#Preprocessing for KNN & SVM

In [19]:
scaler = StandardScaler()

In [20]:
scaler.fit(xy.drop('Label',axis=1))

StandardScaler()

In [21]:
scaled_features = scaler.transform(xy.drop('Label',axis=1))

In [22]:
xy = pd.DataFrame(scaled_features,columns=xy.columns[:-1])
xy['Label']=np.arange(0,4)
xy

,Ant1_real,Ant2_real,Ant1_imag,Ant2_imag,Label
0,-0.225006,-1.289619,0.160145,1.232942,0
1,0.250838,0.004269,1.543993,-0.371181,1
2,-1.406853,1.511813,-1.115124,-1.423544,2
3,1.381021,-0.226463,-0.589014,0.561784,3


#Preprocessing for MLP

In [23]:
xy_NN

,Ant1_real,Ant2_real,Ant1_imag,Ant2_imag,Label
0,0.058622,0.005896,0.328110,0.413490,0
1,0.165617,0.304352,0.942278,-0.305053,1
2,-0.207122,0.652091,-0.237871,-0.776444,2
3,0.419745,0.251130,-0.004377,0.112854,3
0,0.112763,-0.068906,0.353043,0.389358,0
...,...,...,...,...,...
3,0.449977,0.242625,0.062426,0.134471,3
0,0.087510,-0.031945,0.357627,0.400489,0
1,0.194505,0.266510,0.971796,-0.318054,1
2,-0.178235,0.614249,-0.208354,-0.789444,2


In [24]:
'''preprocessing for NN'''
x_NN=xy_NN[['Ant1_real','Ant2_real','Ant1_imag','Ant2_imag']]
y_NN=xy_NN['Label']
'''Labels one hot encoding'''
y_NN=to_categorical(y_NN, num_classes=4)
'''Spliting data to train and validation'''
X_train_NN, X_test_NN, y_train_NN, y_test_NN = train_test_split(x_NN, y_NN, test_size=0.3, random_state=101)

In [25]:
'''Spliting features and labels for SVM & KNN'''
x=xy[['Ant1_real','Ant2_real','Ant1_imag','Ant2_imag']]
y=xy['Label']

In [26]:
xy

,Ant1_real,Ant2_real,Ant1_imag,Ant2_imag,Label
0,-0.225006,-1.289619,0.160145,1.232942,0
1,0.250838,0.004269,1.543993,-0.371181,1
2,-1.406853,1.511813,-1.115124,-1.423544,2
3,1.381021,-0.226463,-0.589014,0.561784,3


In [27]:
'''Spliting data to train and validation'''
#X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.000001, random_state=101)
X_train=x
y_train=y

In [28]:
X_train.shape

(4, 4)

In [29]:
y_train.shape

(4,)

# Building neural network and training

In [30]:
model = Sequential()
model.add(Input(shape=(4)))
model.add(Dense(32, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(rate=0.1))
#model.add(Dense(64, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(4, activation='softmax'))


In [31]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
NN=model.fit(X_train_NN, y_train_NN, batch_size=8, epochs=8, validation_data=(X_test_NN,y_test_NN))

Epoch 1/6
18/18 [==============================] - 3s 73ms/step - loss: 1.3824 - accuracy: 0.2394 - val_loss: 1.3335 - val_accuracy: 0.8387
Epoch 2/6
18/18 [==============================] - 0s 13ms/step - loss: 1.3057 - accuracy: 0.6127 - val_loss: 1.2279 - val_accuracy: 0.8065
Epoch 3/6
18/18 [==============================] - 0s 13ms/step - loss: 1.2044 - accuracy: 0.7183 - val_loss: 1.0644 - val_accuracy: 0.9032
Epoch 4/6
18/18 [==============================] - 0s 18ms/step - loss: 1.0081 - accuracy: 0.8099 - val_loss: 0.8292 - val_accuracy: 0.9677
Epoch 5/6
18/18 [==============================] - 0s 12ms/step - loss: 0.8066 - accuracy: 0.8099 - val_loss: 0.5788 - val_accuracy: 0.9677
Epoch 6/6
18/18 [==============================] - 0s 11ms/step - loss: 0.5868 - accuracy: 0.8732 - val_loss: 0.3841 - val_accuracy: 0.9839


In [32]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                160       
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 4)                 132       
                                                                 
Total params: 4,484
Trainable params: 4,484
Non-trainabl

# Training KNN and SVM model

In [33]:
rbf = svm.SVC(kernel = 'rbf', gamma = 0.5, C = 0.1).fit(X_train, y_train)
poly = svm.SVC(kernel = 'poly', degree = 3, C = 1).fit(X_train, y_train)
knn = KNeighborsClassifier(n_neighbors=1).fit(X_train,y_train)

# Testing models on test dataset 

In [34]:
poly_pred = poly.predict(x_test_svm)
rbf_pred = rbf.predict(x_test_svm)
knn_pred = knn.predict(x_test_svm)

In [35]:
knn_accuracy = accuracy_score(y_test_svm, knn_pred)
knn_f1 = f1_score(y_test_svm, knn_pred, average ='weighted')
print('Accuracy (KNN): ', "%.2f" % (knn_accuracy*100))
print('F1 (KNN): ', "%.2f" % (knn_f1*100))

Accuracy (KNN):  71.55
F1 (KNN):  70.67


In [36]:
poly_accuracy = accuracy_score(y_test_svm, poly_pred)
poly_f1 = f1_score(y_test_svm, poly_pred, average = 'weighted')
print('Accuracy (Polynomial Kernel): ', "%.2f" % (poly_accuracy*100))
print('F1 (Polynomial Kernel): ', "%.2f" % (poly_f1*100))

Accuracy (Polynomial Kernel):  25.45
F1 (Polynomial Kernel):  11.55


In [37]:
rbf_accuracy = accuracy_score(y_test_svm, rbf_pred)
rbf_f1 = f1_score(y_test_svm, rbf_pred, average = 'weighted')
print('Accuracy (RBF Kernel): ', "%.2f" % (rbf_accuracy*100))
print('F1 (RBF Kernel): ', "%.2f" % (rbf_f1*100))

Accuracy (RBF Kernel):  71.55
F1 (RBF Kernel):  70.67


In [38]:
'''Test the model'''
NN_Accuracy = model.evaluate(x_test_nn, y_test_nn)

63/63 [==============================] - 0s 2ms/step - loss: 0.4387 - accuracy: 0.9305


#Choosing K value

In [39]:
'''error_rate = []

for i in range(1,40):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train,y_train)
    pred_i = knn.predict(m1)
    error_rate.append(np.mean(pred_i != n1))'''

'error_rate = []\n\nfor i in range(1,40):\n    knn = KNeighborsClassifier(n_neighbors=i)\n    knn.fit(X_train,y_train)\n    pred_i = knn.predict(m1)\n    error_rate.append(np.mean(pred_i != n1))'

In [40]:
'''plt.figure(figsize=(10,6))
plt.plot(range(1,40),error_rate,color='blue',ls='--',marker='o',markerfacecolor='red',markersize=10)
plt.title('Error Rate vs. K Values')
plt.xlabel('K')
plt.ylabel('Error Rate')'''

"plt.figure(figsize=(10,6))\nplt.plot(range(1,40),error_rate,color='blue',ls='--',marker='o',markerfacecolor='red',markersize=10)\nplt.title('Error Rate vs. K Values')\nplt.xlabel('K')\nplt.ylabel('Error Rate')"